In [2]:
import os
os.chdir("/Users/eric/Projects/sentiment/")

In [2]:
import numpy as np

In [4]:
from app import db, app
from app.emojis import emojis
from app.models import Tweet

/Users/eric/Projects/sentiment/venv/lib/python2.7/site-packages/flask_sqlalchemy/__init__.py:800: UserWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.
  warnings.warn('SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.')


In [4]:
import keras
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM

Using Theano backend.


In [14]:
def data_gen(batch_size=100):
    ids = db.session.query(Tweet.id).all()
    ids = [i[0] for i in ids]
    
    nb_batchs = int(len(ids) / batch_size)
    
    while True:
        np.random.shuffle(ids)
        
        for i in range(nb_batchs):
            
            ids_ss = ids[i*batch_size:(i+1)*batch_size]
            
            tweets = db.session.query(Tweet).filter(Tweet.id.in_(ids_ss)).all()

            xs = [tweet.x for tweet in tweets]
            ys = [tweet.y for tweet in tweets]
            ss = [tweet.sentiment for tweet in tweets]

            x = np.stack(xs)
            y = np.stack(ys)
            s = np.stack(ss)
            
            yield np.stack(xs), [np.stack(ys), np.stack(ss)]

In [15]:
gen = data_gen()

In [18]:
x, y = gen.next()
print x.shape
print y[0].shape
print y[1].shape

(100, 140)
(100, 98)
(100,)


In [8]:
text = Input(shape=(140,))

x = Embedding(input_dim=500, output_dim=10)(text)
x = LSTM(128, return_sequences=True)(x)
x = LSTM(128)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)

emoji = Dense(len(emojis), activation="sigmoid", name="emoji")(x)
sentiment = Dense(1, activation="linear", name="sentiment")(x)

In [9]:
model = Model(text, [emoji, sentiment])

In [10]:
model.compile("RMSprop", 
              loss={'sentiment': "mae", "emoji": "binary_crossentropy"},
              loss_weights={"sentiment":0.2, "emoji": 0.8})

In [11]:
model.input_shape

(None, 140)

In [12]:
model.output_shape

[(None, 98), (None, 1)]

In [19]:
model.fit_generator(gen, samples_per_epoch=1e3, nb_epoch=3)

Epoch 1/3
1000/1000 [==============================] - 13s - loss: 0.3785 - emoji_loss: 0.2456 - sentiment_loss: 0.9097    
Epoch 2/3
1000/1000 [==============================] - 12s - loss: 0.2773 - emoji_loss: 0.1327 - sentiment_loss: 0.8556    
Epoch 3/3
1000/1000 [==============================] - 13s - loss: 0.2434 - emoji_loss: 0.0943 - sentiment_loss: 0.8395    


In [22]:
model.save('./data/model.h5')

In [23]:
m = keras.models.load_model('./data/model.h5')

In [5]:
app.config['BASE_DIR']

'/Users/eric/Projects/sentiment'

In [3]:
from app.ml import *

/Users/eric/Projects/sentiment/venv/lib/python2.7/site-packages/flask_sqlalchemy/__init__.py:800: UserWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.
  warnings.warn('SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.')
Using Theano backend.


In [4]:
sm = SentimentModel()

In [5]:
import json
res = sm.score("hello asshole")
json.dumps(res)

'{"sentiment": 0.6268870234489441, "emoji": {"\\ud83d\\udc7d": 0.0006763793062418699, "\\ud83d\\udc7f": 0.0008735649171285331, "\\ud83d\\udc7e": 0.0006574963335879147, "\\ud83d\\udc79": 0.0009267145651392639, "\\ud83d\\udc7b": 0.003655147971585393, "\\ud83d\\udc7a": 0.00036284560337662697, "\\ud83e\\udd20": 0.0018075841944664717, "\\ud83e\\udd21": 0.0008186119957827032, "\\ud83e\\udd22": 0.00016433425480499864, "\\ud83e\\udd23": 0.0023325912188738585, "\\ud83e\\udd24": 0.0010427703382447362, "\\ud83e\\udd25": 0.0011272174306213856, "\\ud83e\\udd27": 0.001237641554325819, "\\u2620": 0.0005516682867892087, "\\ud83e\\udd10": 0.0031993172597140074, "\\ud83e\\udd11": 0.0012918992433696985, "\\ud83e\\udd12": 0.0011923221172764897, "\\ud83e\\udd13": 0.0018465768080204725, "\\ud83e\\udd14": 0.013511782512068748, "\\ud83e\\udd15": 0.0019174161134287715, "\\ud83e\\udd16": 0.0009297095239162445, "\\ud83e\\udd17": 0.004166895058006048, "\\ud83d\\ude44": 0.02125595696270466, "\\ud83d\\ude41": 0.001

In [3]:
gen = data_gen()

In [4]:
x, y = gen.next()

In [5]:
print x.shape
print y[0].shape
print y[1].shape

(100, 140)
(100, 98)
(100,)


In [7]:
sm.fit()

Epoch 1/10
1000/1000 [==============================] - 14s - loss: 0.2253 - emoji_loss: 0.0805 - sentiment_loss: 0.8047    
Epoch 2/10
1000/1000 [==============================] - 12s - loss: 0.1958 - emoji_loss: 0.0679 - sentiment_loss: 0.7078    
Epoch 3/10
1000/1000 [==============================] - 12s - loss: 0.1944 - emoji_loss: 0.0638 - sentiment_loss: 0.7170    
Epoch 4/10
1000/1000 [==============================] - 12s - loss: 0.1840 - emoji_loss: 0.0582 - sentiment_loss: 0.6873    
Epoch 5/10
1000/1000 [==============================] - 13s - loss: 0.1792 - emoji_loss: 0.0561 - sentiment_loss: 0.6717    
Epoch 6/10
1000/1000 [==============================] - 12s - loss: 0.1800 - emoji_loss: 0.0547 - sentiment_loss: 0.6811    
Epoch 7/10
1000/1000 [==============================] - 12s - loss: 0.1813 - emoji_loss: 0.0529 - sentiment_loss: 0.6948    
Epoch 8/10
1000/1000 [==============================] - 13s - loss: 0.1736 - emoji_loss: 0.0532 - sentiment_loss: 0.6556    


In [26]:
tweet = Tweet("asshole")

In [27]:
x = tweet.x
x = x.reshape(1,-1)
print x

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0  97 115 115 104 111 108 101]]


In [28]:
scores, sentiment = sm._model.predict(x)
scores = [float(s) for s in scores[0, :]]

In [44]:
from app.emojis import emojis

In [52]:
dict(zip(emojis, scores[0, :]))

{u'\u2620': 0.0005553455,
 u'\u2639': 0.00060017704,
 u'\u263a': 0.0033355926,
 u'\U0001f479': 0.00093294395,
 u'\U0001f47a': 0.00036572057,
 u'\U0001f47b': 0.0036794406,
 u'\U0001f47d': 0.00068191515,
 u'\U0001f47e': 0.00066171226,
 u'\U0001f47f': 0.00087931415,
 u'\U0001f480': 0.0041601495,
 u'\U0001f4a9': 0.00253916,
 u'\U0001f600': 0.0060702884,
 u'\U0001f601': 0.011491443,
 u'\U0001f602': 0.19711156,
 u'\U0001f603': 0.0034022275,
 u'\U0001f604': 0.0065536387,
 u'\U0001f605': 0.014090926,
 u'\U0001f606': 0.0051479903,
 u'\U0001f607': 0.0059363567,
 u'\U0001f608': 0.0041555413,
 u'\U0001f609': 0.012023636,
 u'\U0001f60a': 0.035437182,
 u'\U0001f60b': 0.0043066507,
 u'\U0001f60c': 0.010213773,
 u'\U0001f60d': 0.086236186,
 u'\U0001f60e': 0.011207685,
 u'\U0001f60f': 0.016847141,
 u'\U0001f610': 0.0029342778,
 u'\U0001f611': 0.0080885673,
 u'\U0001f612': 0.01243473,
 u'\U0001f613': 0.001427831,
 u'\U0001f614': 0.0088938354,
 u'\U0001f615': 0.0045010862,
 u'\U0001f616': 0.0013939184,
 

In [4]:
from flask import jsonify

In [5]:
res = sm.score('hello')

In [12]:
import json
import simplejson

In [18]:
json.dumps({"a": 1.1000000}, ensure_ascii=False)

'{"a": 1.1}'

In [23]:
type(res['emoji'].values()[1])

numpy.float32